In [ ]:
import numpy as np
import cv2
from tqdm.auto import tqdm

In [ ]:
def read_video(path: str) -> np.array:
    cap = cv2.VideoCapture(path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    for _ in tqdm(range(frame_count), desc='Reading video'):
        ret, frame = cap.read()
        if not ret:
            break

        # converting to grayscale for simplicity
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

    cap.release()
    frames = np.array(frames)

    return frames

In [ ]:
# video_path = '/Volumes/Patrick/Proband06/Logitech HD Pro Webcam C920.avi'
video_path = '/Users/patrick/Desktop/Proband06/Logitech HD Pro Webcam C920.avi'
video_frames = read_video(video_path)

In [ ]:
import matplotlib.pyplot as plt

# Detect the face in the first frame with Viola Jones algorithm
frame1 = video_frames[0].copy()

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

faces = face_cascade.detectMultiScale(frame1, 2.0, 4)

roi = None

for (x, y, w, h) in faces:
    cv2.rectangle(frame1, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # Calculate the chest region based on Cheng_Liu_2023
    chest_x = x
    chest_y = int(y + h + h * 0.7)
    chest_w = w
    chest_h = int(h * 0.5)

    roi = (chest_x, chest_y, chest_w, chest_h)

    cv2.rectangle(frame1, (chest_x, chest_y), (chest_x + chest_w, chest_y + chest_h), (255, 0, 0), 2)
    break

plt.figure(figsize=(5, 5))
plt.imshow(frame1, cmap='gray')
plt.title('Face and chest detection')
plt.show()

## Use the average pixel intensity in the chest region to calculate the breathing rate

In [ ]:
pixels = []

for frame in video_frames:
    # Calculate the average pixel intensity in the chest region
    region = frame[roi[1]:roi[1] + roi[3], roi[0]:roi[0] + roi[2]]
    
    avg_pixel = np.mean(region)
    
    pixels.append(avg_pixel)

In [ ]:
# Plot the pixel intensity over time in an interactive plot
import plotly.express as px

fig = px.line(y=pixels, title='Pixel intensity over time')
fig.show()

In [ ]:
import numpy as np
from scipy.fft import fft

# Normalize the pixel values
pixels_normalized = np.array(pixels) / 255

# Calculate the fast fourier transform of the thorax abdomen data
pixels_fft = fft(pixels_normalized)

# Calculate the frequency
freq = np.fft.fftfreq(len(pixels_fft), 1 / 30)

# Remove the negative frequencies
pixels_fft = pixels_fft[freq > 0]
freq = freq[freq > 0]

In [ ]:
# Only keep the frequencies between 0.1 and 0.6 Hz
bandpass_filter = (freq > 0.09) & (freq < 0.6)

# Remove the negative frequencies
filtered_fft = pixels_fft[bandpass_filter]
filtered_freq = freq[bandpass_filter]

In [ ]:
# Plot the filtered frequency spectrum
fig = px.line(x=filtered_freq, y=np.abs(filtered_fft), title='Filtered Frequency spectrum')
fig.show()

In [ ]:
# Find the frequency with the highest amplitude
max_freq = filtered_freq[np.argmax(np.abs(filtered_fft))]

# Calculate the breathing rate in beats per minute
breathing_rate = max_freq * 60

max_freq, breathing_rate

## Calculate the 

In [ ]:
pixel_intensities = []

for x in range(roi[0], roi[0] + roi[2]):
    for y in range(roi[1], roi[1] + roi[3]):
        for t in range(len(video_frames)):
            pixel_intensities.append(video_frames[t][y, x])
    
pixel_intensities = np.array(pixel_intensities)    

In [ ]:
# Breathing rate matrix
breathing_rate_matrix = np.zeros((roi[3], roi[2]))

# Calculate the fast fourier transform of the thorax abdomen data
for i in range(0, len(pixel_intensities), len(video_frames)):
    pixels_fft = fft(pixel_intensities[i:i + len(video_frames)])

    # Calculate the frequency
    freq = np.fft.fftfreq(len(pixels_fft), 1 / 30)

    # Remove the negative frequencies
    pixels_fft = pixels_fft[freq > 0]
    freq = freq[freq > 0]

    # Only keep the frequencies between 0.1 and 0.6 Hz
    bandpass_filter = (freq > 0.09) & (freq < 0.6)

    # Remove the negative frequencies
    filtered_fft = pixels_fft[bandpass_filter]
    filtered_freq = freq[bandpass_filter]

    # Find the frequency with the highest amplitude
    max_freq = filtered_freq[np.argmax(np.abs(filtered_fft))]

    # Calculate the breathing rate in beats per minute
    breathing_rate = max_freq * 60
    
    # Store the breathing rate in the breathing rate matrix
    x = i // len(video_frames) % roi[2]
    y = i // len(video_frames) // roi[2]
    
    breathing_rate_matrix[y, x] = max_freq

In [ ]:
# Plot the breathing rate matrix
plt.figure(figsize=(10, 10))
plt.imshow(breathing_rate_matrix, cmap='hot', interpolation='nearest')
plt.colorbar()
plt.title('Breathing rate matrix')
plt.show()

In [ ]:
# Calculate the average breathing rate
average_breathing_rate = np.mean(breathing_rate_matrix)
average_breathing_rate, average_breathing_rate * 60